Regional Saturation map

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.transform import Affine
from shapely.geometry import Point
import pickle
from pathlib import Path
import time
import warnings

warnings.filterwarnings('ignore')

print("="*80)
print("REGIONAL SATURATION ANALYSIS - CALCULATION")
print("Operational Onshore Wind Farms Density (35km radius)")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
INPUT_DIR = BASE_DIR
OUTPUT_DIR = BASE_DIR / 'outputs'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

# Output files
SATURATION_COUNT_TIF = OUTPUT_DIR / 'saturation_count.tif'
SATURATION_SCORE_TIF = OUTPUT_DIR / 'saturation_suitability_score.tif'

RADIUS_KM = 35
MAX_SCORE = 10.0
NODATA_VALUE = -9999
CRS_EPSG = 27700

# Step 1: Load reference grid
print("\nStep 1: Loading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref = pickle.load(f)

rows, cols = ref['shape']
pixel_size = ref['pixel_size']
x_spacing, y_spacing = pixel_size
extent = ref['extent']

x_min = extent[0]
y_max = extent[3]

transform = Affine(
    pixel_size[0], 0, extent[0],
    0, -abs(pixel_size[1]), extent[3]
)

print(f"Shape: {rows} x {cols}")
print(f"Pixel size: {x_spacing:.2f} x {y_spacing:.2f} m")
print(f"Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Step 2: Load operational wind farms
print("\nStep 2: Loading operational onshore wind farms (>50 MW)")

df = pd.read_csv(INPUT_DIR / 'repd_data.csv', encoding='latin-1', low_memory=False)
print(f"✓ Total records loaded: {len(df):,}")

# Filter for onshore wind
onshore_wind = df[df['Technology Type'] == 'Wind Onshore'].copy()
print(f"✓ Onshore wind farms: {len(onshore_wind):,}")

# Convert capacity to numeric
onshore_wind['Installed Capacity (MWelec)'] = pd.to_numeric(
    onshore_wind['Installed Capacity (MWelec)'], 
    errors='coerce'
)

# Filter for capacity > 50 MW
onshore_wind = onshore_wind[onshore_wind['Installed Capacity (MWelec)'] > 50].copy()
print(f"✓ Onshore wind farms > 50 MW: {len(onshore_wind):,}")

# Filter for OPERATIONAL ONLY
def is_operational(status):
    if pd.isna(status):
        return False
    status = str(status).strip()
    return status == 'Operational'

onshore_wind['Is_Operational'] = onshore_wind['Development Status (short)'].apply(is_operational)
operational_wind = onshore_wind[onshore_wind['Is_Operational'] == True].copy()
print(f"✓ Operational wind farms: {len(operational_wind):,}")

# Step 3: Create GeoDataFrame of operational wind farms
print("\nStep 3: Processing spatial coordinates of operational farms")

operational_wind = operational_wind[
    (operational_wind['X-coordinate'].notna()) & 
    (operational_wind['Y-coordinate'].notna())
].copy()

operational_wind['X-coordinate'] = pd.to_numeric(operational_wind['X-coordinate'], errors='coerce')
operational_wind['Y-coordinate'] = pd.to_numeric(operational_wind['Y-coordinate'], errors='coerce')

operational_wind = operational_wind[
    (operational_wind['X-coordinate'].notna()) & 
    (operational_wind['Y-coordinate'].notna())
].copy()

geometry = [Point(xy) for xy in zip(operational_wind['X-coordinate'], operational_wind['Y-coordinate'])]
wind_gdf = gpd.GeoDataFrame(operational_wind, geometry=geometry, crs='EPSG:27700')

print(f"✓ Operational wind farms with valid coordinates: {len(wind_gdf)}")
print(f"  Total operational capacity: {wind_gdf['Installed Capacity (MWelec)'].sum():.0f} MW")

# Step 4: Calculate saturation (count of farms within 35km)
print(f"\nStep 4: Calculating saturation (farms within {RADIUS_KM} km of each cell)")
print("This will take several minutes...")

saturation_count = np.zeros((rows, cols), dtype=np.float32)
radius_meters = RADIUS_KM * 1000

# Get all operational farm coordinates as numpy array
farm_coords = np.array([[geom.x, geom.y] for geom in wind_gdf.geometry])

start_time = time.time()

# Process each cell
for r in range(rows):
    for c in range(cols):
        # Calculate cell center coordinates
        x_center = x_min + (c + 0.5) * x_spacing
        y_center = y_max - (r + 0.5) * y_spacing
        
        # Calculate distances to all operational farms
        distances = np.sqrt((farm_coords[:, 0] - x_center)**2 + (farm_coords[:, 1] - y_center)**2)
        
        # Count farms within radius
        count = np.sum(distances <= radius_meters)
        saturation_count[r, c] = count
    
    # Progress update
    if (r + 1) % 10 == 0:
        elapsed = time.time() - start_time
        progress = (r + 1) / rows * 100
        print(f"  Progress: {progress:.1f}% (row {r+1}/{rows}) - {elapsed:.0f}s elapsed")

elapsed = time.time() - start_time
print(f"\nSaturation calculation complete in {elapsed:.0f} seconds")

# Step 5: Apply UK land mask
print("\nStep 5: Applying UK land mask")
uk_boundary = gpd.read_file(INPUT_DIR / 'UK_ITL1_Boundaries.shp')
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')

land_mask = np.zeros((rows, cols), dtype=bool)
uk_dissolved = uk_boundary.dissolve()
uk_geom = uk_dissolved.geometry.iloc[0]

for r in range(rows):
    for c in range(cols):
        y = y_max - r * y_spacing - y_spacing / 2
        x = x_min + c * x_spacing + x_spacing / 2
        if uk_geom.contains(Point(x, y)):
            land_mask[r, c] = True

saturation_count[~land_mask] = np.nan

valid_cells = (~np.isnan(saturation_count)).sum()
print(f"Valid land cells: {valid_cells:,}")

# Step 6: Calculate percentile-based scores (DECILE METHOD)
print("\nStep 6: Calculating saturation scores using percentile-based scaling")

valid_saturation = saturation_count[~np.isnan(saturation_count)]

# Calculate decile thresholds (0, 10, 20, ..., 90, 100 percentiles)
deciles = np.percentile(valid_saturation, range(0, 101, 10))
print(f"\nDecile thresholds (farms within {RADIUS_KM}km):")
for i, threshold in enumerate(deciles):
    print(f"  {i*10}th percentile: {threshold:.1f} farms")

# Calculate scores: lowest saturation (0th decile) → score 10
#                   highest saturation (100th decile) → score 0
saturation_score = np.zeros_like(saturation_count, dtype=np.float32)

valid_mask = ~np.isnan(saturation_count)

for r in range(rows):
    for c in range(cols):
        if valid_mask[r, c]:
            count = saturation_count[r, c]
            
            # Find percentile rank
            percentile_rank = np.searchsorted(deciles, count) * 10
            percentile_rank = min(percentile_rank, 100)
            
            # Invert: low saturation = high score
            score = MAX_SCORE * (1 - percentile_rank / 100)
            saturation_score[r, c] = score

saturation_score[~land_mask] = np.nan

# Step 7: Statistics
print("\nStep 7: Statistics")

valid_scores = saturation_score[~np.isnan(saturation_score)]

print(f"\nSaturation count statistics:")
print(f"  Min: {valid_saturation.min():.0f} farms")
print(f"  Max: {valid_saturation.max():.0f} farms")
print(f"  Mean: {valid_saturation.mean():.1f} farms")
print(f"  Median: {np.median(valid_saturation):.1f} farms")

print(f"\nScore statistics:")
print(f"  Min: {valid_scores.min():.2f}")
print(f"  Max: {valid_scores.max():.2f}")
print(f"  Mean: {valid_scores.mean():.2f}")
print(f"  Median: {np.median(valid_scores):.2f}")

print(f"\nScore distribution:")
for threshold in [10, 8, 6, 4, 2, 0]:
    count = np.sum(valid_scores >= threshold)
    pct = count / len(valid_scores) * 100
    print(f"  Score ≥ {threshold:2d}: {count:7,} cells ({pct:5.1f}%)")

# Step 8: Save GeoTIFF files
print("\nStep 8: Saving GeoTIFF files")

metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': NODATA_VALUE,
    'width': cols,
    'height': rows,
    'count': 1,
    'crs': f'EPSG:{CRS_EPSG}',
    'transform': transform,
    'compress': 'lzw'
}

# Save saturation count
count_save = np.where(np.isnan(saturation_count), NODATA_VALUE, saturation_count)
with rasterio.open(SATURATION_COUNT_TIF, 'w', **metadata) as dst:
    dst.write(count_save.astype('float32'), 1)
print(f"✓ Saved: {SATURATION_COUNT_TIF}")

# Save saturation score
score_save = np.where(np.isnan(saturation_score), NODATA_VALUE, saturation_score)
with rasterio.open(SATURATION_SCORE_TIF, 'w', **metadata) as dst:
    dst.write(score_save.astype('float32'), 1)
print(f"✓ Saved: {SATURATION_SCORE_TIF}")

# Step 9: Regional analysis
print("\nStep 9: Regional saturation analysis")

# Identify regions (approximate)
scotland_mask = land_mask & (saturation_count >= 0)
scotland_north = scotland_mask.copy()
scotland_north[:int(rows*0.4), :] = False  # Rough approximation

scotland_saturation = saturation_count[scotland_north & ~np.isnan(saturation_count)]
england_saturation = saturation_count[land_mask & (saturation_count >= 0) & ~scotland_north & ~np.isnan(saturation_count)]

if len(scotland_saturation) > 0:
    print(f"\nScotland region:")
    print(f"  Mean saturation: {scotland_saturation.mean():.1f} farms")
    print(f"  Mean score: {saturation_score[scotland_north & ~np.isnan(saturation_score)].mean():.2f}")

if len(england_saturation) > 0:
    print(f"\nEngland/Wales region:")
    print(f"  Mean saturation: {england_saturation.mean():.1f} farms")
    print(f"  Mean score: {saturation_score[land_mask & ~scotland_north & ~np.isnan(saturation_score)].mean():.2f}")

print("\n" + "="*80)
print("SATURATION CALCULATION COMPLETE")
print("="*80)
print(f"\nKey findings:")
print(f"  • {len(wind_gdf)} operational farms used for analysis")
print(f"  • Saturation range: 0-{valid_saturation.max():.0f} farms within {RADIUS_KM} km")
print(f"  • Scores calculated using percentile-based decile method")
print(f"  • Low saturation areas receive high scores (favor development)")
print(f"  • High saturation areas receive low scores (discourage further development)")
print("\nOutput files:")
print(f"  • {SATURATION_COUNT_TIF.name} (farm counts)")
print(f"  • {SATURATION_SCORE_TIF.name} (suitability scores 0-10)")
print("\nNext step: Run visualization script to generate maps")
print("="*80)

REGIONAL SATURATION ANALYSIS - CALCULATION
Operational Onshore Wind Farms Density (35km radius)

Step 1: Loading reference grid
Shape: 235 x 131
Pixel size: 5000.00 x 5000.00 m
Extent: X[0, 655000], Y[10000, 1185000]

Step 2: Loading operational onshore wind farms (>50 MW)
✓ Total records loaded: 13,524
✓ Onshore wind farms: 2,657
✓ Onshore wind farms > 50 MW: 315
✓ Operational wind farms: 59

Step 3: Processing spatial coordinates of operational farms
✓ Operational wind farms with valid coordinates: 59
  Total operational capacity: 6173 MW

Step 4: Calculating saturation (farms within 35 km of each cell)
This will take several minutes...
  Progress: 4.3% (row 10/235) - 0s elapsed
  Progress: 8.5% (row 20/235) - 0s elapsed
  Progress: 12.8% (row 30/235) - 0s elapsed
  Progress: 17.0% (row 40/235) - 0s elapsed
  Progress: 21.3% (row 50/235) - 0s elapsed
  Progress: 25.5% (row 60/235) - 0s elapsed
  Progress: 29.8% (row 70/235) - 0s elapsed
  Progress: 34.0% (row 80/235) - 0s elapsed
  P

Regional Saturation suitability score map

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pickle
from pathlib import Path
from shapely.geometry import Point
import warnings

warnings.filterwarnings('ignore')

print("="*80)
print("REGIONAL SATURATION VISUALIZATION")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
INPUT_DIR = BASE_DIR
OUTPUT_DIR = BASE_DIR / 'outputs'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

# Input files (from calculation script)
SATURATION_COUNT_TIF = OUTPUT_DIR / 'saturation_count.tif'
SATURATION_SCORE_TIF = OUTPUT_DIR / 'saturation_suitability_score.tif'

# Output files
SATURATION_MAP_PNG = OUTPUT_DIR / 'map_saturation_analysis.png'

RADIUS_KM = 35
NODATA_VALUE = -9999
CRS_EPSG = 27700

# Step 1: Load reference grid
print("\nStep 1: Loading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref = pickle.load(f)

extent = ref['extent']
print(f"Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Step 2: Load saturation rasters
print("\nStep 2: Loading saturation rasters")

if not SATURATION_COUNT_TIF.exists():
    print(f"ERROR: {SATURATION_COUNT_TIF} not found!")
    print("Please run saturation_calculate.py first!")
    exit(1)

if not SATURATION_SCORE_TIF.exists():
    print(f"ERROR: {SATURATION_SCORE_TIF} not found!")
    print("Please run saturation_calculate.py first!")
    exit(1)

with rasterio.open(SATURATION_COUNT_TIF) as src:
    saturation_count = src.read(1)
    saturation_count[saturation_count == NODATA_VALUE] = np.nan
    land_mask = ~np.isnan(saturation_count)

with rasterio.open(SATURATION_SCORE_TIF) as src:
    saturation_score = src.read(1)
    saturation_score[saturation_score == NODATA_VALUE] = np.nan

valid_saturation = saturation_count[~np.isnan(saturation_count)]
valid_scores = saturation_score[~np.isnan(saturation_score)]

print(f"✓ Loaded saturation count: {len(valid_saturation):,} cells")
print(f"✓ Loaded saturation scores: {len(valid_scores):,} cells")
print(f"  Count range: {valid_saturation.min():.0f} - {valid_saturation.max():.0f} farms")
print(f"  Score range: {valid_scores.min():.2f} - {valid_scores.max():.2f}")

# Step 3: Load operational wind farms for plotting
print("\nStep 3: Loading operational wind farms for visualization")

df = pd.read_csv(INPUT_DIR / 'repd_data.csv', encoding='latin-1', low_memory=False)
onshore_wind = df[df['Technology Type'] == 'Wind Onshore'].copy()
onshore_wind['Installed Capacity (MWelec)'] = pd.to_numeric(
    onshore_wind['Installed Capacity (MWelec)'], 
    errors='coerce'
)
onshore_wind = onshore_wind[onshore_wind['Installed Capacity (MWelec)'] > 50].copy()

def is_operational(status):
    if pd.isna(status):
        return False
    return str(status).strip() == 'Operational'

onshore_wind['Is_Operational'] = onshore_wind['Development Status (short)'].apply(is_operational)
operational_wind = onshore_wind[onshore_wind['Is_Operational'] == True].copy()

operational_wind = operational_wind[
    (operational_wind['X-coordinate'].notna()) & 
    (operational_wind['Y-coordinate'].notna())
].copy()

operational_wind['X-coordinate'] = pd.to_numeric(operational_wind['X-coordinate'], errors='coerce')
operational_wind['Y-coordinate'] = pd.to_numeric(operational_wind['Y-coordinate'], errors='coerce')

operational_wind = operational_wind[
    (operational_wind['X-coordinate'].notna()) & 
    (operational_wind['Y-coordinate'].notna())
].copy()

geometry = [Point(xy) for xy in zip(operational_wind['X-coordinate'], operational_wind['Y-coordinate'])]
wind_gdf = gpd.GeoDataFrame(operational_wind, geometry=geometry, crs='EPSG:27700')

print(f"✓ Loaded {len(wind_gdf)} operational wind farms")

# Step 4: Load UK boundary
print("\nStep 4: Loading UK boundary")
uk_boundary = gpd.read_file(INPUT_DIR / 'UK_ITL1_Boundaries.shp')
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')
print("✓ UK boundary loaded")

# Step 5: Create visualization map
print("\nStep 5: Creating visualization map (dual panel)...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12), dpi=300)

# Left panel: Saturation count
data_plot1 = np.where(land_mask, saturation_count, np.nan)
im1 = ax1.imshow(
    data_plot1,
    extent=[extent[0], extent[1], extent[2], extent[3]],
    origin='upper',
    cmap='YlOrRd',
    interpolation='nearest',
    vmin=0,
    vmax=np.percentile(valid_saturation, 95)  # Cap at 95th percentile
)

uk_boundary.boundary.plot(ax=ax1, color='black', linewidth=0.8, zorder=10)
wind_gdf.plot(ax=ax1, color='blue', markersize=5, alpha=0.6, zorder=15, label='Operational farms')

ax1.set_xlabel('Easting (km)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Northing (km)', fontsize=12, fontweight='bold')
ax1.set_title(f'Regional Saturation\n(Operational farms within {RADIUS_KM} km)', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.2)

cbar1 = plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
cbar1.set_label(f'Number of farms within {RADIUS_KM} km', fontsize=11, fontweight='bold')

# Right panel: Saturation score
data_plot2 = np.where(land_mask, saturation_score, np.nan)
im2 = ax2.imshow(
    data_plot2,
    extent=[extent[0], extent[1], extent[2], extent[3]],
    origin='upper',
    cmap='RdYlGn',  # Red (high saturation/low score) to Green (low saturation/high score)
    interpolation='nearest',
    vmin=0,
    vmax=10
)

uk_boundary.boundary.plot(ax=ax2, color='black', linewidth=0.8, zorder=10)

ax2.set_xlabel('Easting (km)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Northing (km)', fontsize=12, fontweight='bold')
ax2.set_title('Saturation Suitability Score\n(Percentile-based, 0-10)', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.2)

cbar2 = plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
cbar2.set_label('Suitability Score (higher = less saturated)', fontsize=11, fontweight='bold')

# Format axes (convert meters to km)
def format_km(value, tick_number):
    return f'{int(value/1000)}'

for ax in [ax1, ax2]:
    ax.xaxis.set_major_formatter(FuncFormatter(format_km))
    ax.yaxis.set_major_formatter(FuncFormatter(format_km))

fig.suptitle(
    f'Regional Wind Farm Saturation Analysis ({RADIUS_KM} km radius)\n' +
    f'Based on {len(wind_gdf)} Operational Onshore Wind Farms (>50 MW)',
    fontsize=16,
    fontweight='bold',
    y=0.98
)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(SATURATION_MAP_PNG, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ Saved: {SATURATION_MAP_PNG}")
plt.close()

print("\n" + "="*80)
print("VISUALIZATION COMPLETE")
print("="*80)
print(f"\nOutput file:")
print(f"  • {SATURATION_MAP_PNG.name}")
print("\nMap details:")
print(f"  • Left panel: Raw saturation count (farms within {RADIUS_KM} km)")
print(f"  • Right panel: Suitability score (0-10, inverted)")
print(f"  • Blue dots: {len(wind_gdf)} operational wind farms")
print(f"  • Resolution: Dual panel, 20x12 inches, 300 DPI")
print("="*80)

REGIONAL SATURATION VISUALIZATION

Step 1: Loading reference grid
Extent: X[0, 655000], Y[10000, 1185000]

Step 2: Loading saturation rasters
✓ Loaded saturation count: 9,758 cells
✓ Loaded saturation scores: 9,758 cells
  Count range: 0 - 13 farms
  Score range: 0.00 - 10.00

Step 3: Loading operational wind farms for visualization
✓ Loaded 59 operational wind farms

Step 4: Loading UK boundary
✓ UK boundary loaded

Step 5: Creating visualization map (dual panel)...
✓ Saved: D:\BENV0093\outputs\map_saturation_analysis.png

VISUALIZATION COMPLETE

Output file:
  • map_saturation_analysis.png

Map details:
  • Left panel: Raw saturation count (farms within 35 km)
  • Right panel: Suitability score (0-10, inverted)
  • Blue dots: 59 operational wind farms
  • Resolution: Dual panel, 20x12 inches, 300 DPI
